In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import time
from pymongo import MongoClient
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg

ModuleNotFoundError: No module named 'seaborn'

Connecting to a MongoDB server running at host "localhost" on port 27017, the connect to 'air-quality' database

In [ ]:
client = MongoClient(host='localhost', port=27017)
db = client['air-quality']
dar = db['dar-es-salaam']

In [ ]:
dar.count_documents({})

Determine the numbers assigned to all the sensor sites in the Dar es Salaam collection.

In [ ]:
sites = dar.distinct("metadata.site")
print(dar.count_documents({'metadata.site': 23}))
print(dar.count_documents({'metadata.site': 11}))
sites

Determine which site in the Dar es Salaam collection has the most sensor readings

In [ ]:
result = dar.aggregate([
    {'$group': {'_id': '$metadata.site', 'count': {'$count': {}}}}
])
readings_per_site = list(result)
readings_per_site

Wrangle function extracts the PM2.5 readings from the site that has the most total readings in the Dar es Salaam collection.
Localize reading time stamps to the timezone for "Africa/Dar_es_Salaam".
Remove all outlier PM2.5 readings that are above 100.
Resample the data to provide the mean PM2.5 reading for each hour.
Impute any missing values using the forward-fill method.

In [ ]:
def wrangle(collection):
    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read results into DataFrame
    df = pd.DataFrame(list(results)).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")

    # Remove outliers
    df = df[df["P2"] < 100]

    # Resample and forward-fill
    y = df['P2'].resample('1H').mean().fillna(method='ffill')

    return y

In [ ]:
y = wrangle(dar)
y.head()

Creates a time series plot of the readings in y.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

y.plot(xlabel='Date', ylabel='PM2.5 Level', title='Dar es Salaam PM2.5 Levels', ax=ax);

plt.savefig("images/3-5-5.png", dpi=150)


Plots the rolling average of the readings in y use a window size of 168 (the number of hours in a week).

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

y.rolling(168).mean().plot(ax=ax, ylabel='PM2.5', title='Weekly Rolling Average');

plt.savefig("images/3-5-6.png", dpi=150)

Creates an ACF plot for the data in y, labelling the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient"

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

plot_acf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");
plt.title('Dar es Salaam PM2.5')

plt.savefig("images/3-5-7.png", dpi=150)

Creates a PACF plot for the data in y, labelling the x-axis as "Lag [hours]" and the y-axis as "Correlation Coefficient"

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

plot_pacf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient");
plt.title('Dar es Salaam PM2.5')

plt.savefig("images/3-5-8.png", dpi=150)

Split y into training and test sets, with the first 90% of the data in the training set, remaining 10% in the test set.

In [ ]:
cutoff_test = int(len(y) * 0.9)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Establishing the baseline mean absolute error for the model:

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

Uses an AutoReg model to predict PM2.5 readings. lags from 1 to 30. 
Each time a new model is trained, its mean absolute error is calculated and the result appended to the list maes.
Results stored in the Series 'mae_series'.

In [ ]:
#Create range to test different lags
p_params = range(1, 31)

# Create empty list to hold mean absolute error scores
maes = []

# Iterate through all values of p in `p_params`
for p in p_params:
    # Build model
    model = AutoReg(y_train, lags=p).fit()

    # Make predictions on training data, dropping null values caused by lag
    y_pred = model.predict().dropna()

    # Calculate mean absolute error for training data vs predictions
    mae = mean_absolute_error(y_train.iloc[p:], y_pred)

    # Append `mae` to list `maes`
    maes.append(mae)

# Put list `maes` into Series with index `p_params`
mae_series = pd.Series(maes, name="mae", index=p_params)

# Inspect head of Series
mae_series.head()

In [ ]:
mae_series

Determine what value for p provides the best performance. Then build and train best_model using the best hyperparameter value

In [ ]:
best_p = mae_series.idxmin()
best_model = AutoReg(y_train, lags=best_p).fit()

Calculates the training residuals for best_model and assign the result to y_train_resid

In [ ]:
y_train_resid = model.resid
y_train_resid.name = "residuals"
y_train_resid.head()

Create a histogram of y_train_resid, labelling the x-axis as "Residuals" and the y-axis as "Frequency"

In [ ]:
plt.savefig("images/3-5-14.png", dpi=150)

Create an ACF plot for y_train_resid. Be sure to label the x-axis as "Lag [hours]" and y-axis as "Correlation Coefficient"

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y_train_resid.plot(ylabel='Correlation Coefficient', xlabel='Lag [hours]', title='Dar es Salaam, Training Residuals ACF', ax=ax)

plt.savefig("images/3-5-15.png", dpi=150)

Performing walk-forward validation for the model for the entire test set y_test. Stores the model's predictions in the Series y_pred_wfv. Index of the series is "timestamp".

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()
best_p = mae_series.idxmin()
for i in range(len(y_test)):
    model = AutoReg(history, lags=best_p).fit()
    next_pred = model.forecast()
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

Puts the values for y_test and y_pred_wfv into the DataFrame df_pred_test
Then plot df_pred_test using plotly express.

In [ ]:
df_pred_test = pd.DataFrame({'y_test': y_test,'y_pred_wfv': y_pred_wfv}, index=y_test.index)
fig = px.line(df_pred_test)
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)

fig.write_image("images/3-5-18.png", scale=1, height=500, width=700)

fig.show()